<a href="https://colab.research.google.com/github/jalew188/PeptDeep-HLA/blob/master/nbs/HLA1_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning of sample-specific HLA-I models

In [1]:
%pip install git+https://github.com/MannLabs/PeptDeep-HLA.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MannLabs/PeptDeep-HLA.git to /tmp/pip-req-build-jgz00cbh
  Running command git clone -q https://github.com/MannLabs/PeptDeep-HLA.git /tmp/pip-req-build-jgz00cbh


In [2]:
%pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import gdown

fasta_url = "https://drive.google.com/file/d/1MKGRBpzvmMW0l_hdPESo3j26EWd_yi8l/view?usp=share_link"
fasta = 'irtfusion.fasta'

gdown.download(fasta_url, fasta, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1MKGRBpzvmMW0l_hdPESo3j26EWd_yi8l
To: /content/irtfusion.fasta
100%|██████████| 174/174 [00:00<00:00, 209kB/s]


'irtfusion.fasta'

#### Load training HLA peptides

It could be a tsv/csv/txt file containing sample-specific HLA-I peptides. It must contain 'sequence' column. 

In [4]:
import pandas as pd
train_seq_df = pd.DataFrame({
    'sequence': [
        'ACDEFGHIKLMNPQ',
        'ACDEFGHI',
        'ACDEFGHIK',
        'EFGHIKLMNPQ',
        'AHIKLMNPQ',
    ]
})
train_seq_df['nAA'] = train_seq_df.sequence.str.len()
train_seq_df

,sequence,nAA
0,ACDEFGHIKLMNPQ,14
1,ACDEFGHI,8
2,ACDEFGHIK,9
3,EFGHIKLMNPQ,11
4,AHIKLMNPQ,9


#### Initialize the model and load the pretrained model

In [5]:
from peptdeep_hla.HLA_class_I import HLA_Class_I_Classifier
model = HLA_Class_I_Classifier(
    fasta_files=[fasta]
)
model.get_parameter_num()

1669697

In [6]:
from peptdeep_hla.HLA_class_I import pretrained_HLA1
model.load(pretrained_HLA1)
pretrained_HLA1

'/usr/local/lib/python3.8/dist-packages/peptdeep_hla/pretrained_models/HLA1_IEDB.pt'

#### Train by the training peptides

The non-HLA peptides are automatically sampled from the fasta file as the negative training data.

In [7]:
model.train(
    train_seq_df, 
    epoch=4, warmup_epoch=2, 
    verbose=True
)

[Training] Epoch=1, lr=5e-05, loss=1.7478619366884232
[Training] Epoch=2, lr=0.0001, loss=1.1243337094783783
[Training] Epoch=3, lr=5e-05, loss=0.8942193537950516
[Training] Epoch=4, lr=0.0, loss=0.5556356459856033


#### Predict HLA-I peptides from fasta

In [8]:
hla_df = model.predict_from_proteins(prob_threshold=0.7)
hla_df

100%|██████████| 1/1 [00:01<00:00,  1.14s/it]


,start_pos,end_pos,nAA,HLA_prob_pred,sequence
0,53,61,8,0.771491,GLDAKTPV
1,54,62,8,0.840000,LDAKTPVI
2,66,74,8,0.831190,YEYRVEAT
3,67,75,8,0.894142,EYRVEATF
4,22,30,8,0.790146,FIIDPGGV
...,...,...,...,...,...
186,78,92,14,0.717524,ESNAKTPVITGAPY
187,79,93,14,0.858710,SNAKTPVITGAPYE
188,80,94,14,0.910569,NAKTPVITGAPYEY
189,81,95,14,0.742917,AKTPVITGAPYEYR
